In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotVisibleException, InvalidArgumentException

In [128]:
chromedriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver)
select = driver.find_elements_by_css_selector
select_one = driver.find_element_by_css_selector
#start linkedIn with a fresh page, no one is logged in
driver.get('https://www.linkedin.com/')

In [2]:
def sign_in(login, password, driver):#if on the main linked in with the sign in on top right
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    driver.get('https://www.linkedin.com/')
    try:
        sign_in = select_one('input#login-email')
        sign_in.click()
        sign_in.send_keys(login)
        password_in = select_one('input#login-password')
        password_in.click()
        password_in.send_keys(password)
        login = select_one('input#login-submit')
        login.click()
    except NoSuchElementException:       
        try:
            sign_in = select_one('a.nav__button-secondary')
            sign_in.click()
            try:
                input_name = select_one('input#username')
                input_name.send_keys(login)
                input_pwd = select_one('input#password')
                input_pwd.send_keys(password)
                login_action = select_one('div.login__form_action_container ')
                login_action.click()
            except NoSuchElementException:
                    driver.get(sign_in.get_attribute('href'))
                    input_name = select_one('input#username')
                    input_name.click()
                    input_name.send_keys(login)
                    input_password = select_one('input#password')
                    input_password.click()
                    input_password.send_keys(password)
                    login_btn = select_one('button.btn__primary--large.from__button--floating')
                    login_btn.click()
        except NoSuchElementException:
            try:
                email_input = select_one('input.login-email')
                email_input.click()
                email_input.send_keys(input)
                password_input = select_one('input.login-password')
                password_input.click()
                password_input.send_keys(password)
                submit = select_one('#login-submit')
                submit.click()

            except:
                print('nope, didnt work')
    return(driver)


In [96]:
login = 'keri@multerer.com'
password = 'jfdkfjadluivjdkk5'

In [156]:
sign_in(login,password,driver)

nope, didnt work


<selenium.webdriver.chrome.webdriver.WebDriver (session="6afbc6b88be958924a33714b487d8eaa")>

# Getting list of conversations on the left and then clicking through them to capture the full message thread

In [90]:
def get_message_urls(driver, df):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector

    message_thread_urls = []

    unordered_list = select_one('ul.msg-conversations-container__conversations-list')
    convo_list = unordered_list.find_elements_by_css_selector('a')
    for c in convo_list:
        message_thread_urls.append(c.get_attribute('href'))
    #adding the first # of messages to match the index of the df as there were some inconsistencies in the number of message urls and threads scraped.
    df['message_url'] = message_thread_urls[:len(df)]
    return(df)


In [198]:
def get_full_messages(driver,df):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector

    convo_list = select('li.msg-conversation-listitem')
    print(len(convo_list))
    for convo in convo_list:
        try:
            convo.click()
            try:
                message = select_one('div.msg-s-message-list').text
                message_list = message.split('\n')
                time.sleep(1)
                try:
                    message_container = select_one('div.global-title-container')
                    profile_url = message_container.find_element_by_css_selector('a').get_attribute('href')
                except NoSuchElementException:
                    profile_url = 'unable to get'
                if message_list[0] == 'Recruiter':
                    df = df.append({'id': message_list[0], 'message': message_list, 'date': message_list[1], 'profile_url':profile_url}, ignore_index=True)
                elif '2' in message_list[0]:
                    if len(message_list) >= 4:
                        df = df.append({'id': message_list[3], 'message': message_list, 'date': message_list[0], 'profile_url':profile_url}, ignore_index=True)
                    else:
                        print(message_list)
                        df = df.append({'id': message_list[1][:-21], 'message': message_list, 'date': message_list[0], 'profile_url':profile_url}, ignore_index=True)
                else:
                    df = df.append({'id': message_list[0], 'message': message_list, 'date': message_list[5], 'profile_url':profile_url}, ignore_index=True)
            except NoSuchElementException:
                df = df.append({'id': '' , 'message' : 'advertisement', 'date': '', 'profile_url':'none'}, ignore_index=True)
        except ElementNotVisibleException:
            return(df)
    return(df)
    

In [199]:
def get_messages(driver):
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    df = pd.DataFrame(columns=['id','message','date','profile_url'])
    #accessing the messages once logged in
    message_btn = select_one('span#messaging-tab-icon')
    message_btn.click()
    #get full message text, name and date of person
    messages_df = get_full_messages(driver,df)
    #getting url to messages
    add_message_urls_df = get_message_urls(driver,messages_df)
    return(add_message_urls_df)

# Getting the stats for visits

In [200]:
def get_viewer_info(driver):
    driver.get('https://www.linkedin.com/')
    viewers = {}
    select = driver.find_elements_by_css_selector
    select_one = driver.find_element_by_css_selector
    #go to profile page
    profile = select_one('li#profile-nav-item')
    profile.click()
    time.sleep(1)
    viewer_page = select_one('span.ember-view')
    viewer_page.click()
    time.sleep(2)
    
    #get the profile view information
    viewer_list = select('div.me-wvmp-viewer-card.display-flex.ember-view')
    for viewer in viewer_list:
        try:
            viewer_url = viewer.find_element_by_css_selector('a').get_attribute('href')
            viewer_name = viewer.find_element_by_css_selector('span.me-wvmp-viewer-card__name-text').text
            viewers[viewer_name] = viewer_url
        except NoSuchElementException:
            pass
    viewer_df = pd.DataFrame.from_dict(viewers, orient='index')
    viewer_df.reset_index(inplace=True)
    viewer_df.columns=['name','profile_url']
    return(viewer_df)

In [10]:
# def get_ssindex(driver):
#     select = driver.find_elements_by_css_selector
#     select_one = driver.find_element_by_css_selector
#     social_index_df = pd.DataFrame(columns=['ninety_day_views','social_score','weekly_totals'])
#     driver.get('https://www.linkedin.com/')
#     views = get_ninetyday_views(driver)
#     #accessing your social selling index: Must be logged in then go to https://www.linkedin.com/sales/ssi
#     driver.get('https://www.linkedin.com/sales/ssi')
#     body = select_one('article#main')
#     wrappers = body.find_element_by_css_selector('div#current-barchart-wrapper')
#     score_list = wrappers.text.split('\n')
#     labels = score_list[4:8]
#     scores = score_list[-4:]
#     #put this into the dataframe
#     scores_tuple_list = list(zip(labels,scores))
#     weekly_totals = select_one('div#overtime-linechart-accessible')
#     time.sleep(5)
#     table = weekly_totals.find_element_by_css_selector('table')
#     body = table.find_element_by_css_selector('tbody')
#     data = body.find_elements_by_css_selector('td')
#     data_table_list = []
#     for d in data:
#         data_table_list.append(d.get_attribute('innerHTML'))
#     #put this into the dataframe
#     social_index = social_index_df.append({'ninety_day_views': views,
#                     'social_score': scores_tuple_list,
#                     'weekly_totals': data_table_list}, 
#                     ignore_index=True)
#     return(social_index)

In [201]:
def run_scrape(login, password):
    chromedriver = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chromedriver)
    sign_in(login, password, driver)
    #messages = get_messages(driver)
    messages = get_messages(driver)
    filename = f'{login}'+'.messages.csv'
    messages.to_csv(filename,index=False)

    #viewer information
    viewer_df = get_viewer_info(driver)
    viewerfilename = f'{login}'+'viewerinfo.csv'
    viewer_df.to_csv(viewerfilename,index=False)
    
    

In [12]:
chromedriver = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver)
driver = sign_in(login, password,driver)

In [202]:
run_scrape(login, password)

20
['JUL 25, 2018', 'Daniel Jang is now a connection.']


In [203]:
!ls

geckodriver.log                   messages_csv_example.xlsx
keri@multerer.com.messages.csv    views_graph_example.xlsx
keri@multerer.comsocial_score.csv workbook.ipynb
keri@multerer.comviewerinfo.csv


In [204]:
df = pd.read_csv('keri@multerer.com.messages.csv')

In [205]:
df

,id,message,date,profile_url,message_url
0,Faisal Zahid,"['Faisal Zahid', 'Faisal Zahid', '· 1st', 'Tec...",Faisal Zahid sent the following message at 2:3...,https://www.linkedin.com/in/faisal-zahid-280118b/,https://www.linkedin.com/messaging/thread/6495...
1,NaN,advertisement,NaN,none,https://www.linkedin.com/messaging/thread/6514...
2,Yiran Sherry,"['Yiran Sherry', 'Yiran Sherry', 'Enterprise T...",View Yiran’s profile,https://www.linkedin.com/in/yiransherry/,https://www.linkedin.com/messaging/thread/6505...
3,NaN,advertisement,NaN,none,https://www.linkedin.com/messaging/thread/6495...
4,NaN,advertisement,NaN,none,https://www.linkedin.com/messaging/thread/6478...
5,Recruiter,"['Recruiter', 'Oct 17, 2018', 'Sophia Georgant...","Oct 17, 2018",https://www.linkedin.com/in/sophia-georgantoni...,https://www.linkedin.com/messaging/thread/6458...
6,Elena Poughia,"['Elena Poughia', 'Elena Poughia', 'MD at Data...",View Elena’s profile,https://www.linkedin.com/in/elenapoughia/,https://www.linkedin.com/messaging/thread/6451...
7,NaN,advertisement,NaN,none,https://www.linkedin.com/messaging/thread/6444...
8,Charlie Redding,"['AUG 11, 2018', 'Charlie Redding sent the fol...","AUG 11, 2018",unable to get,https://www.linkedin.com/messaging/thread/6409...
9,Daniel Jang,"['JUL 25, 2018', 'Daniel Jang is now a connect...","JUL 25, 2018",https://www.linkedin.com/in/daniel-jang/,https://www.linkedin.com/messaging/thread/6427...


In [206]:
viewers = pd.read_csv('keri@multerer.comviewerinfo.csv')

In [207]:
viewers

,name,profile_url
0,Ha Tran,https://www.linkedin.com/in/ha-t-tran/
1,Abhi Banerjee,https://www.linkedin.com/in/abhi-banerjee/
2,Helene Thomas,https://www.linkedin.com/in/helene-thomas-7600...
3,John Tlapa,https://www.linkedin.com/in/johntlapa/
4,Keri Mallari,https://www.linkedin.com/in/kerixyz/
5,Niels Hanson,https://www.linkedin.com/in/niels-hanson-46ab1...


In [ ]:
!rm 'keri@multerer.com.messages.csv'
!rm 'keri@multerer.comsocial_score.csv'

In [ ]:
!ls